Import **Libraries**

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
# Import Tokenizer and pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!pip install nltk
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
print(tf.__version__)

In [ ]:

import nltk
nltk.download('all')
nltk.download('stopwords') # download stopwords
nltk.download('wordnet') 

In [ ]:
MAXLIN = 500
VOCAB_SIZE = 1000
BATCH_SIZE = 64

**Useful** Function

In [ ]:
lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split()
                    if not word in stopwords_eng])
    txt = txt.lower()
    return txt

In [ ]:
def preprocessing(X):
  x = X.apply(lambda x: clean_txt(x))
  x = tokenizer.texts_to_sequences(x)
  return keras.preprocessing.sequence.pad_sequences(x, MAXLIN)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=['y', 'x'])
train_data.head()

In [ ]:
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=['y', 'x'])
test_data.head()

In [ ]:
#categorized each of  variables into numbers 
y_train = train_data['y'].astype('category').cat.codes
y_test  = test_data['y'].astype('category').cat.codes
y_train[:10]


In [ ]:
x_train = train_data['x'].astype('category').cat.codes
x_test  = test_data['x'].astype('category').cat.codes
x_train[:10]

In [ ]:
stopwords_eng = set(stopwords.words('english'))
len(stopwords_eng)

x_train = train_data['x'].apply(lambda x: clean_txt(x))
x_train[:5]


**Preprocessing**

Create the Tokenizer and define an out of vocabulary token


In [ ]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token="<OOV>")

Tokenize the words

In [ ]:
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
sequences[:5]

In [ ]:


Matrix_sequences =pad_sequences(sequences,MAXLIN)
Matrix_sequences[:5]


**Model**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE,50),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='relu')
                             
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']

)
model.summary()

In [ ]:

result = model.fit(Matrix_sequences, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2)

**Test**

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
